# Adversarial Search: Playing Dots and Boxes


## Instructions

Total Points: Undegraduates 100, graduate students 110

Complete this notebook and submit it. The notebook needs to be a complete project report with your implementation, documentation including a short discussion of how your implementation works and your design choices, and experimental results (e.g., tables and charts with simulation results) with a short discussion of what they mean. Use the provided notebook cells and insert additional code and markdown cells as needed.

## Introduction

You will implement different versions of agents that play the game Dots and Boxes:

> "Dots and Boxes is a pencil-and-paper game for two players. The game starts with an empty grid of dots. Usually two players take turns adding a single horizontal or vertical line between two unjoined adjacent dots. A player who completes the fourth side of a 1x1 box earns one point and takes another turn. A point is typically recorded by placing a mark that identifies the player in the box, such as an initial. The game ends when no more lines can be placed. The winner is the player with the most points. The board may be of any size grid." (see [Dots and Boxes on Wikipedia](https://en.wikipedia.org/wiki/Dots_and_Boxes))

You can play Dots and Boxes [here](https://www.math.ucla.edu/~tom/Games/dots&boxes.html).

## Task 1: Defining the Search Problem [10 point]

Define the components of the search problem associated with this game:

* Initial state
* Actions
* Transition model
* Test for the terminal state
* Utility for terminal states

In [1]:
# Your code/answer goes here.
# Representation of an initial state (empty board)
def initial_state(rows=4, cols=4):
    """
    rows, cols = number of dot rows and dot columns (as used elsewhere in the notebook)
    Board representation:
      - 'size': (rows, cols)  # number of dots
      - 'lines': dict()       # keys: (orientation, r, c) where orientation in {'h','v'}
      - 'boxes': dict()       # keys: (r,c) box coordinates, value: +1 or -1 for owner
      - 'next_player': +1     # +1 or -1 indicating who moves next
    """
    return {'size': (rows, cols), 'lines': {}, 'boxes': {}, 'next_player': +1}

# Actions: any not-yet-drawn line; action format: (orientation, row, col)
def actions(board):
    rows, cols = board['size']
    acts = []
    # horizontal lines: rows * (cols-1)
    for r in range(rows):
        for c in range(cols-1):
            a = ('h', r, c)
            if a not in board['lines']:
                acts.append(a)
    # vertical lines: (rows-1) * cols
    for r in range(rows-1):
        for c in range(cols):
            a = ('v', r, c)
            if a not in board['lines']:
                acts.append(a)
    return acts

# Transition model: result(board, action, player) -> new board (deep copy).
# If the action completes one or more boxes, those boxes are assigned to player and player moves again.
import copy
def result(board, action, player=None):
    if player is None:
        player = board.get('next_player', +1)
    b = copy.deepcopy(board)
    ori, r, c = action
    # place the line
    b['lines'][(ori, r, c)] = True

    # helper to check if a box at (br,bc) is completed
    def box_completed(bd, br, bc):
        # four sides: top ('h', br, bc), bottom ('h', br+1, bc), left ('v', br, bc), right ('v', br, bc+1)
        sides = [
            ('h', br, bc),
            ('h', br+1, bc),
            ('v', br, bc),
            ('v', br, bc+1)
        ]
        return all(s in bd['lines'] for s in sides)

    rows, cols = b['size']
    claimed = 0
    # check boxes that might be affected by the placed line (at most 2)
    candidate_boxes = []
    if ori == 'h':
        # horizontal at (r,c) is top of box (r,c) and bottom of box (r-1,c)
        if r < rows-1:
            candidate_boxes.append((r, c))
        if r-1 >= 0:
            candidate_boxes.append((r-1, c))
    else:  # 'v'
        # vertical at (r,c) is left of box (r,c) and right of box (r,c-1)
        if c < cols-1:
            candidate_boxes.append((r, c))
        if c-1 >= 0:
            candidate_boxes.append((r, c-1))

    for br, bc in candidate_boxes:
        if 0 <= br < rows-1 and 0 <= bc < cols-1:
            if box_completed(b, br, bc) and (br, bc) not in b['boxes']:
                b['boxes'][(br, bc)] = player
                claimed += 1

    # update next_player: same player if claimed >=1, otherwise switch
    b['next_player'] = player if claimed >= 1 else -player
    return b

# Terminal test: board is terminal when number of drawn lines == total possible lines
def total_possible_lines(board):
    rows, cols = board['size']
    return rows * (cols - 1) + (rows - 1) * cols

def is_terminal(board):
    return len(board['lines']) >= total_possible_lines(board)

# Utility for terminal states:
# return integer score = (sum of boxes owned by +1) - (sum of boxes owned by -1)
# positive means advantage for player +1, negative advantage for player -1
def utility(board):
    # only meaningful at terminal states, but defined generally
    return sum(board['boxes'].values())

- initial_state:
Bảng trống, không có đường kẻ, không có ô nào được yêu cầu; biểu diễn được hiển thị bởi initial_state().

- actions:
Tập hợp các đường kẻ có thể vẽ; mỗi hành động = (hướng, hàng, cột) với hướng trong {'h','v'}. Xem actions(board).

- transition_model:
Xác định: việc vẽ một đường kẻ có thể hoàn thành các ô; các ô đã hoàn thành sẽ được gán cho người chơi đang di chuyển và người chơi đó sẽ di chuyển tiếp nếu có bất kỳ ô nào được hoàn thành. Xem result(board, action, player).

- terminal_test:
Trò chơi kết thúc khi tất cả các đường kẻ có thể đã được vẽ (không có hành động nào khả dụng). Xem is_terminal(board).

- utility:
Tại trạng thái terminal, utility = hiệu số giữa các ô đã sở hữu: +1 ô trừ -1 ô (sử dụng utility(board)).

How big is the state space? Give an estimate and explain it.

In [2]:
# Your code/ answer goes here.
# Estimate state-space and game-tree size for Dots-and-Boxes
import math

def dots_and_boxes_size(rows=5, cols=5):
    """
    rows, cols = number of dot rows and dot columns (default 5x5 dots -> 4x4 boxes)
    Prints concise estimates and short explanation.
    """
    L = rows * (cols - 1) + (rows - 1) * cols   # total possible lines (edges)
    B = (rows - 1) * (cols - 1)                # number of boxes
    # Naive upper bound on states (each line present/absent): 2^L
    states_by_lines = 2 ** L
    # If we also (naively) allow each completed box to have two possible owners,
    # an even looser upper bound is 2^(L) * 2^B = 2^(L+B).
    states_with_box_owners = 2 ** (L + B)
    # include next-player bit
    states_with_nextplayer = 2 ** (L + B + 1)

    # Rough estimate of distinct play sequences (upper bound): any permutation of the L lines
    # -> L! possible complete sequences (game tree of move orders is enormous).
    sequences = math.factorial(L)

    print(f"dots: {rows}x{cols}  lines L = {L}, boxes B = {B}")
    print()
    print("State-space (upper bounds):")
    print(f"  - subsets of lines only: 2^L = {states_by_lines:.3e}")
    print(f"  - add naive box-owner choices: 2^(L+B) = {states_with_box_owners:.3e}")
    print(f"  - include next-player bit: 2^(L+B+1) = {states_with_nextplayer:.3e}")
    print()
    print("Game-tree (upper bounds):")
    print(f"  - permutations of all moves (complete orders): L! = {sequences:.3e}")
    print("  - the full game tree (all partial histories) is even larger (sum of P(L,k) for k=0..L).")
    print()
    print("Short explanation:")
    print("  - Every line can be present/absent -> 2^L possible line-sets (naive).")
    print("  - Box ownership depends on move order, so counting owners multiplies possibilities (loose bound 2^B).")
    print("  - The number of possible complete play sequences is at most L! (each ordering of lines).")
    print("  - Conclusion: even for a 5x5-dot board (L=40) the state space and game tree are astronomically large")
    print("    (e.g., 2^40 ≈ 1.1e12, 40! ≈ 8.2e47), so exhaustive minimax is infeasible for full board without pruning/heuristics.")

# Example for the standard 5x5-dot board
dots_and_boxes_size(5,5)

dots: 5x5  lines L = 40, boxes B = 16

State-space (upper bounds):
  - subsets of lines only: 2^L = 1.100e+12
  - add naive box-owner choices: 2^(L+B) = 7.206e+16
  - include next-player bit: 2^(L+B+1) = 1.441e+17

Game-tree (upper bounds):
  - permutations of all moves (complete orders): L! = 8.159e+47
  - the full game tree (all partial histories) is even larger (sum of P(L,k) for k=0..L).

Short explanation:
  - Every line can be present/absent -> 2^L possible line-sets (naive).
  - Box ownership depends on move order, so counting owners multiplies possibilities (loose bound 2^B).
  - The number of possible complete play sequences is at most L! (each ordering of lines).
  - Conclusion: even for a 5x5-dot board (L=40) the state space and game tree are astronomically large
    (e.g., 2^40 ≈ 1.1e12, 40! ≈ 8.2e47), so exhaustive minimax is infeasible for full board without pruning/heuristics.


How big is the game tree that minimax search will go through? Give an estimate and explain it.

In [3]:
# Your code/ answer goes here.
# Estimate game-tree size that minimax must explore (upper bounds and intuition)
import math

def game_tree_estimate(rows=5, cols=5):
    """
    rows, cols = number of dot rows and dot columns.
    Estimates:
      - L = total number of possible line moves (depth)
      - leaves <= L! (every permutation of moves)
      - total nodes (all partial sequences) ≈ e * L! (approx)
      - naive uniform-branching estimate using average branching ~ (L+1)/2
    Prints concise numeric approximations (log10 for huge numbers).
    """
    L = rows * (cols - 1) + (rows - 1) * cols
    b_avg = (L + 1) / 2.0  # very rough average branching factor
    # log10 of L! via lgamma to avoid huge intermediates
    log10_fact = math.lgamma(L + 1) / math.log(10)
    log10_leaves = log10_fact
    # approx total nodes ≈ e * L! -> log10_nodes ≈ log10(e) + log10(L!)
    log10_total_nodes = math.log10(math.e) + log10_fact
    # naive uniform-branching estimate: b_avg^L
    log10_naive = L * math.log10(b_avg) if b_avg > 0 else float('-inf')

    print(f"dots: {rows}x{cols}  (L = {L} possible moves / tree depth)")
    print()
    print("Upper bounds (orders of magnitude):")
    print(f"  - Leaves (upper bound) <= L!  -> log10(L!) = {log10_leaves:.3f}")
    print(f"    -> L! ≈ 10^{log10_leaves:.3f}")
    print(f"  - Total nodes (all partial histories) ≈ e * L!  -> log10 ≈ {log10_total_nodes:.3f}")
    print(f"    -> nodes ≈ 10^{log10_total_nodes:.3f}")
    print()
    print("Naive branching estimate (using avg branching ≈ (L+1)/2):")
    print(f"  - avg branching b ≈ {b_avg:.2f}; log10(b^L) = {log10_naive:.3f}")
    print(f"    -> b^L ≈ 10^{log10_naive:.3f}")
    print()
    print("Interpretation:")
    print("  - The game-tree grows factorially in L (≈ L!), so even small increases in board size")
    print("    produce astronomically larger trees.")
    print("  - Alpha-beta pruning with excellent move ordering can reduce nodes dramatically,")
    print("    but worst-case complexity remains infeasible for standard boards; use heuristics/cutoffs.")
    print()

# Examples
game_tree_estimate(3,3)   # 3x3 dots -> small example (L=12)
game_tree_estimate(4,4)   # 4x4 dots -> medium (L=24)
game_tree_estimate(5,5)   # 5x5 dots -> standard example (L=40)

dots: 3x3  (L = 12 possible moves / tree depth)

Upper bounds (orders of magnitude):
  - Leaves (upper bound) <= L!  -> log10(L!) = 8.680
    -> L! ≈ 10^8.680
  - Total nodes (all partial histories) ≈ e * L!  -> log10 ≈ 9.115
    -> nodes ≈ 10^9.115

Naive branching estimate (using avg branching ≈ (L+1)/2):
  - avg branching b ≈ 6.50; log10(b^L) = 9.755
    -> b^L ≈ 10^9.755

Interpretation:
  - The game-tree grows factorially in L (≈ L!), so even small increases in board size
    produce astronomically larger trees.
  - Alpha-beta pruning with excellent move ordering can reduce nodes dramatically,
    but worst-case complexity remains infeasible for standard boards; use heuristics/cutoffs.

dots: 4x4  (L = 24 possible moves / tree depth)

Upper bounds (orders of magnitude):
  - Leaves (upper bound) <= L!  -> log10(L!) = 23.793
    -> L! ≈ 10^23.793
  - Total nodes (all partial histories) ≈ e * L!  -> log10 ≈ 24.227
    -> nodes ≈ 10^24.227

Naive branching estimate (using avg branchin

## Task 2: Game Environment and Random Agent [30 point]

You need to think about a data structure to represent the board meaning he placed lines and who finished what box. There are many options. Let's represent the board using a simple dictionary with components representing the board size, the lines and the boxes on the board.

**Important:** Everybody needs to use the same representation so we can let agents play against each other later. 

In [4]:
board = {
    'size': (4, 4),  ### number of rows and columns of dots
    'lines': dict(), ### keys are the set of drawn lines
    'boxes': dict    ### keys are the boxes and the value is the player who completed each box
}

def draw_line(board, orientation, row, col):
    """
    Place a line on an exiting board.
       
    Parameters
    ----------
    board: dict
        the board
    orientation: str
        either 'h' or 'v' for horizontal or vertical
    row, col: int
        index of the starting dot for the line (starting with 0)
    
    """
    
    if orientation not in ['h', 'v']:
        return False
        
    if row < 0 or col < 0:
        return False
        
    if row >= board['size'][0] + (orientation == 'v') or col >= board['size'][1] + (orientation == 'h'):
        return False
        
    if (orientation, row, col) in board['lines']:
        return False
            
    board["lines"][(orientation, row, col)] = True
    return True
    

print(draw_line(board, "h", 1, 1))
print(draw_line(board, "v", 1, 1))

# this should not work
print(draw_line(board, "h", 1, 1))

board

True
True
False


{'size': (4, 4),
 'lines': {('h', 1, 1): True, ('v', 1, 1): True},
 'boxes': dict}

Write code to display the board. 

**Bonus point: Post your visualization code with an example output to the discussion board. The best visualization will earn you bonus participation points in this class.**

In [5]:
# Your code/ answer goes here.
def display_board(board):
    """
    ASCII display for the dots-and-boxes board.
    - board['size'] = (rows, cols)  # number of dot rows and columns
    - board['lines'] keys: ('h'|'v', r, c)
    - board['boxes'] keys: (r,c) -> +1 / -1
    """
    rows, cols = board['size']
    owners = board.get('boxes', {})
    out_lines = []
    # iterate dot-rows
    for r in range(rows):
        # horizontal line row with dots
        dot_row = []
        for c in range(cols):
            dot_row.append('●')
            if c < cols - 1:
                dot_row.append('───' if ('h', r, c) in board['lines'] else '   ')
        out_lines.append(''.join(dot_row))
        # verticals + box contents (between dot rows)
        if r < rows - 1:
            box_row = []
            for c in range(cols):
                box_row.append('│' if ('v', r, c) in board['lines'] else ' ')
                if c < cols - 1:
                    val = owners.get((r, c), 0)
                    if val == +1:
                        box_row.append(' R ')
                    elif val == -1:
                        box_row.append(' Y ')
                    else:
                        box_row.append('   ')
            out_lines.append(''.join(box_row))
    print('\n'.join(out_lines))

# Example usage
board_example = initial_state(4, 4)
# draw a small square in top-left
draw_line(board_example, 'h', 0, 0)
draw_line(board_example, 'v', 0, 0)
draw_line(board_example, 'h', 1, 0)
draw_line(board_example, 'v', 0, 1)
# assign top-left box to player +1 to show marking
board_example['boxes'][(0, 0)] = +1

display_board(board_example)


●───●   ●   ●
│ R │        
●───●   ●   ●
             
●   ●   ●   ●
             
●   ●   ●   ●


Implement helper functions for:

* The transition model $result(s, a)$.
* The utility function $utility(s)$.
* Check for terminal states $terminal(s)$.
* A check for available actions in each state $actions(s)$.

__Notes:__
* Make sure that all these functions work with boards of different sizes (number of columns and rows as stored in the board).
* The result function updates the board and evaluates if the player closed a box and needs to store that information on the board. Add elements of the form `(row,col): player` to the board dictionary. `row` and `col` are the coordinates for the box and `player` is +1 or -1 representing the player. For example `(0,0): -1` means that the top-left box belongs to the other player. 
* _Important:_ Remember that a player goes again after she completes a box!

In [6]:
# Your code/ answer goes here.
def _box_sides(br, bc):
    return [('h', br, bc),
            ('h', br + 1, bc),
            ('v', br, bc),
            ('v', br, bc + 1)]

def box_completed(bd, br, bc):
    return all(s in bd['lines'] for s in _box_sides(br, bc))

def result(board, action, player=None):
    if player is None:
        player = board.get('next_player', +1)
    b = copy.deepcopy(board)
    ori, r, c = action
    b['lines'][(ori, r, c)] = True
    rows, cols = b['size']
    claimed = 0
    candidate_boxes = []
    if ori == 'h':
        if r < rows - 1:
            candidate_boxes.append((r, c))
        if r - 1 >= 0:
            candidate_boxes.append((r - 1, c))
    else:  # 'v'
        if c < cols - 1:
            candidate_boxes.append((r, c))
        if c - 1 >= 0:
            candidate_boxes.append((r, c - 1))
    for br, bc in candidate_boxes:
        if 0 <= br < rows - 1 and 0 <= bc < cols - 1:
            if box_completed(b, br, bc) and (br, bc) not in b['boxes']:
                b['boxes'][(br, bc)] = player
                claimed += 1
    b['next_player'] = player if claimed >= 1 else -player
    return b

def utility(board):
    return sum(board['boxes'].values())

Implement an agent that plays randomly. Make sure the agent function receives as the percept the board and returns a valid action. Use an agent function definition with the following signature (arguments):

`def random_player(board, player = None): ...`

The argument `player` is used for agents that do not store what side they are playing. The value passed on by the environment should be 1 ot -1 for playerred and yellow, respectively.  See [Experiments section for tic-tac-toe](https://nbviewer.org/github/mhahsler/CS7320-AI/blob/master/Games/tictactoe_and_or_tree_search.ipynb#Experiments) for an example.

In [7]:
# Your code/ answer goes here.
import random

def random_player(board, player=None):
    acts = actions(board)
    if not acts:
        return None
    return random.choice(acts)

Let two random agents play against each other 1000 times. Look at the [Experiments section for tic-tac-toe](https://nbviewer.org/github/mhahsler/CS7320-AI/blob/master/Games/tictactoe_and_or_tree_search.ipynb#Experiments) to see how the environment uses the agent functions to play against each other.

How often does each player win? Is the result expected?

In [8]:
# Your code/ answer goes here.

def play_one_game(agent_p, agent_q, rows=4, cols=4, starter=+1):
    """Play a single game between agent_p (+1) and agent_q (-1)."""
    board = initial_state(rows, cols)
    board['next_player'] = starter
    agents = {+1: agent_p, -1: agent_q}
    while not is_terminal(board):
        player = board.get('next_player', +1)
        action = agents[player](board, player)
        if action is None:
            break
        board = result(board, action, player)
    score = utility(board)  # positive => +1 wins, negative => -1 wins, 0 => draw
    if score > 0:
        return +1
    if score < 0:
        return -1
    return 0

def tournament_random(n_games=1000, rows=4, cols=4, verbose=True):
    wins = {+1: 0, -1: 0, 0: 0}
    for i in range(n_games):
        # alternate starting player to be fair
        starter = +1 if (i % 2 == 0) else -1
        winner = play_one_game(random_player, random_player, rows, cols, starter)
        wins[winner] += 1
    if verbose:
        total = n_games
        print(f"Played {total} games (alternating starter).")
        print(f"Player +1 wins: {wins[+1]} ({wins[+1]/total:.2%})")
        print(f"Player -1 wins: {wins[-1]} ({wins[-1]/total:.2%})")
        print(f"Draws: {wins[0]} ({wins[0]/total:.2%})")
    return wins

# Run the experiment
tournament_random(1000, rows=4, cols=4)


Played 1000 games (alternating starter).
Player +1 wins: 506 (50.60%)
Player -1 wins: 494 (49.40%)
Draws: 0 (0.00%)


{1: 506, -1: 494, 0: 0}

## Task 3: Minimax Search with Alpha-Beta Pruning [30 points]

### Implement the search starting.

Implement the search starting from a given board and specifying the player and put it into an agent function.
You can use code from the [tic-tac-toe example](https://nbviewer.org/github/mhahsler/CS7320-AI/blob/master/Games/tictactoe_alpha_beta_tree_search.ipynb).

__Notes:__ 
* Make sure that all your agent functions have a signature consistent with the random agent above.
* The search space for larger board may be too large. You can experiment with smaller boards.
* Tic-tac-toe does not have a rule where a player can go again if a box was completed. You need to adapt the tree search to reflect that rule.

In [49]:
# Your code/ answer goes here.
import math
import random

def is_terminal(board):
    """Kiểm tra xem trò chơi đã kết thúc chưa."""
    return get_winner(board) is not None or all(cell != ' ' for row in board for cell in row)

def get_winner(board):
    """Trả về người thắng ('X' hoặc 'O') hoặc None nếu chưa có ai thắng."""
    size = len(board)
    lines = []

    # Hàng và cột
    for i in range(size):
        lines.append(board[i])  # hàng
        lines.append([board[j][i] for j in range(size)])  # cột

    # Đường chéo
    lines.append([board[i][i] for i in range(size)])
    lines.append([board[i][size - 1 - i] for i in range(size)])

    for line in lines:
        if all(cell == 'X' for cell in line):
            return 'X'
        if all(cell == 'O' for cell in line):
            return 'O'
    return None

def opponent(player):
    return 'O' if player == 'X' else 'X'

def count_potential_wins(board, player):
    """Đếm số dòng/cột/chéo mà player có thể thắng."""
    count = 0
    size = len(board)
    lines = []

    for i in range(size):
        lines.append(board[i])  # hàng
        lines.append([board[j][i] for j in range(size)])  # cột

    lines.append([board[i][i] for i in range(size)])  # chéo chính
    lines.append([board[i][size - 1 - i] for i in range(size)])  # chéo phụ

    for line in lines:
        if all(cell == player or cell == ' ' for cell in line):
            count += 1
    return count


def evaluate(board, player):
    winner = get_winner(board)
    if winner == player:
        return 10
    elif winner is not None:
        return -10
    else:
        return count_potential_wins(board, player) - count_potential_wins(board, opponent(player))


def get_valid_moves(board):
    """Trả về danh sách các nước đi hợp lệ dưới dạng (row, col)."""
    moves = []
    for i in range(len(board)):
        for j in range(len(board[i])):
            if board[i][j] == ' ':
                moves.append((i, j))
    return moves

def minimax(board, depth, alpha, beta, maximizing_player, player):
    """Thuật toán minimax với alpha-beta pruning."""
    opponent = 'O' if player == 'X' else 'X'

    if depth == 0 or is_terminal(board):
        return evaluate(board, player), None

    best_move = None
    if maximizing_player:
        max_eval = -math.inf
        for move in get_valid_moves(board):
            i, j = move
            board[i][j] = player
            eval, _ = minimax(board, depth - 1, alpha, beta, False, player)
            board[i][j] = ' '
            if eval > max_eval:
                max_eval = eval
                best_move = move
            alpha = max(alpha, eval)
            if beta <= alpha:
                break
        return max_eval, best_move
    else:
        min_eval = math.inf
        for move in get_valid_moves(board):
            i, j = move
            board[i][j] = opponent
            eval, _ = minimax(board, depth - 1, alpha, beta, True, player)
            board[i][j] = ' '
            if eval < min_eval:
                min_eval = eval
                best_move = move
            beta = min(beta, eval)
            if beta <= alpha:
                break
        return min_eval, best_move

def alphabeta_agent(board, player):
    """Agent sử dụng minimax với alpha-beta pruning."""
    _, move = minimax(board, depth=5, alpha=-math.inf, beta=math.inf, maximizing_player=True, player=player)
    if move is None:
        # Nếu không tìm được nước đi tốt, chọn ngẫu nhiên
        move = random.choice(get_valid_moves(board))
    return move


Hàm alphabeta_agent(board, player) là agent chính ta cần dùng.

Hàm này tương thích với agent ngẫu nhiên: nhận vào board và player, trả về một nước đi (row, col).

Ta nên điều điều chỉnh depth=5 để phù hợp với kích thước bàn cờ (ví dụ: 3x3 thì depth=5 là đủ), vì số lượng trạng thái có thể tăng rất nhanh (đặc biệt với bàn cờ lớn hơn), nên giới hạn depth giúp:

- Giảm thời gian tính toán.

- Tránh bị treo máy hoặc chạy quá lâu.

- Cân bằng giữa độ chính xác và hiệu suất.

- Nếu dùng bàn cờ 3x3, có thể để depth = 9 (tối đa số ô trống), nhưng nếu dùng bàn 4x4 hoặc lớn hơn, nên giảm depth xuống (ví dụ 4 hoặc 5) để tránh quá tải.

Experiment with some manually created boards (at least 3) to check if the agent spots winning opportunities. Discuss the results.

Mỗi bàn cờ là một ma trận 3x3 với 'X', 'O', và ' ' (ô trống). Agent sẽ là người chơi 'X'.

In [47]:
# Your code/ answer goes here.
print("Board 1: Cơ hội thắng ngay lập tức, kỳ vọng agent chọn nước (0,2)")
board1 = [
    ['X', 'X', ' '],
    ['O', 'O', ' '],
    [' ', ' ', ' ']
]
move1 = alphabeta_agent(board1, 'X')
print("Agent chọn:", move1)

print("\nBoard 2: Chặn đối thủ thắng, kỳ vọng agent chọn nước (0,2)")
board2 = [
    ['O', 'O', ' '],
    ['X', ' ', ' '],
    [' ', ' ', 'X']
]
move2 = alphabeta_agent(board2, 'X')
print("Agent chọn:", move2)

print("\nBoard 3: Ưu tiên chiến thắng hơn là chặn, kỳ vọng agent chọn nước (0,1)")
board3 = [
    ['X', ' ', 'X'],
    ['O', 'O', ' '],
    [' ', ' ', ' ']
]
move3 = alphabeta_agent(board3, 'X')
print("Agent chọn:", move3)

print("\nBoard 4: Tình huống phức tạp, kỳ vọng agent chọn nước (0,2) tạo thế fork")
board4 = [
    ['X', ' ', ' '],
    [' ', 'O', ' '],
    [' ', ' ', 'X']
]
move4 = alphabeta_agent(board4, 'X')
print("Board 4 - Agent chọn:", move4)



Board 1: Cơ hội thắng ngay lập tức, kỳ vọng agent chọn nước (0,2)
Agent chọn: (0, 2)

Board 2: Chặn đối thủ thắng, kỳ vọng agent chọn nước (0,2)
Agent chọn: (0, 2)

Board 3: Ưu tiên chiến thắng hơn là chặn, kỳ vọng agent chọn nước (0,1)
Agent chọn: (0, 1)

Board 4: Tình huống phức tạp, kỳ vọng agent chọn nước (0,2) tạo thế fork
Board 4 - Agent chọn: (0, 1)


### Nhận xét tổng quát

- Agent hoạt động tốt trong các tình huống thắng trực tiếp hoặc phòng thủ rõ ràng.
- Tuy nhiên, trong các tình huống chiến thuật nâng cao như “fork”, agent chưa thể hiện được sự tinh tế nếu không được hỗ trợ bởi hàm đánh giá phù hợp.
- Việc cải tiến hàm `evaluate` để nhận diện các thế cờ tiềm năng (như fork) sẽ giúp agent trở nên thông minh hơn và ra quyết định giống con người hơn.



How long does it take to make a move? Start with a smaller board make the board larger. What is the largest board you can solve?

In [50]:
# Your code/ answer goes here.
import time

def create_empty_board(size):
    return [[' ' for _ in range(size)] for _ in range(size)]

sizes = [3, 4, 5, 6, 7]
for size in sizes:
    board = create_empty_board(size)
    start_time = time.time()
    move = alphabeta_agent(board, 'X')
    end_time = time.time()
    print(f"Board {size}x{size} → Agent chọn: {move} | Thời gian: {end_time - start_time:.4f} giây")


Board 3x3 → Agent chọn: (1, 1) | Thời gian: 0.0787 giây
Board 4x4 → Agent chọn: (0, 0) | Thời gian: 0.5000 giây
Board 5x5 → Agent chọn: (0, 0) | Thời gian: 3.2348 giây
Board 6x6 → Agent chọn: (0, 0) | Thời gian: 12.6566 giây
Board 7x7 → Agent chọn: (0, 0) | Thời gian: 53.6827 giây


Agent hoạt động rất nhanh với bàn cờ 3x3 và 4x4.

Với bàn 5x5 trở lên, số lượng trạng thái tăng theo cấp số nhân → thời gian xử lý tăng mạnh.

Với Minimax thuần (không có heuristic nâng cao), kích thước tối đa có thể giải hợp lý là 5x5.

Để mở rộng hơn, cần:

- Giảm depth

- Dùng hàm đánh giá tốt hơn

- Áp dụng kỹ thuật như Monte Carlo Tree Search hoặc heuristic pruning

### Move ordering

Starting the search with better moves will increase the efficiency of alpha-beta pruning. Describe and implement a simple move ordering strategy. 

Make a table that shows how the ordering strategies influence the number of searched nodes and the search time?

In [ ]:
# Your code/ answer goes here.
def order_moves(board, moves):
    """Sắp xếp nước đi theo độ ưu tiên: trung tâm > góc > cạnh."""
    size = len(board)
    center = (size // 2, size // 2)

    def score(move):
        i, j = move
        if move == center:
            return 3
        elif (i == 0 or i == size - 1) and (j == 0 or j == size - 1):
            return 2  # góc
        else:
            return 1  # cạnh

    return sorted(moves, key=score, reverse=True)

# Viết lại hàm minimax để sử dụng order_moves
def minimax1(board, depth, alpha, beta, maximizing_player, player):
    """Thuật toán minimax với alpha-beta pruning."""
    opponent = 'O' if player == 'X' else 'X'

    if depth == 0 or is_terminal(board):
        return evaluate(board, player), None

    best_move = None
    if maximizing_player:
        max_eval = -math.inf
        for move in order_moves(board, get_valid_moves(board)):
            i, j = move
            board[i][j] = player
            eval, _ = minimax1(board, depth - 1, alpha, beta, False, player)
            board[i][j] = ' '
            if eval > max_eval:
                max_eval = eval
                best_move = move
            alpha = max(alpha, eval)
            if beta <= alpha:
                break
        return max_eval, best_move
    else:
        min_eval = math.inf
        for move in order_moves(board, get_valid_moves(board)):
            i, j = move
            board[i][j] = opponent
            eval, _ = minimax1(board, depth - 1, alpha, beta, True, player)
            board[i][j] = ' '
            if eval < min_eval:
                min_eval = eval
                best_move = move
            beta = min(beta, eval)
            if beta <= alpha:
                break
        return min_eval, best_move
    
# Đo hiệu quả move ordering
import time

def count_nodes_minimax(board, depth, alpha, beta, maximizing_player, player, node_counter):
    """Minimax có đếm số node duyệt."""
    node_counter[0] += 1
    opponent = 'O' if player == 'X' else 'X'

    if depth == 0 or is_terminal(board):
        return evaluate(board, player), None

    best_move = None
    moves = get_valid_moves(board)

    # bật/tắt dòng dưới để so sánh
    moves = order_moves(board, moves)  

    if maximizing_player:
        max_eval = -math.inf
        for move in moves:
            i, j = move
            board[i][j] = player
            eval, _ = count_nodes_minimax(board, depth - 1, alpha, beta, False, player, node_counter)
            board[i][j] = ' '
            if eval > max_eval:
                max_eval = eval
                best_move = move
            alpha = max(alpha, eval)
            if beta <= alpha:
                break
        return max_eval, best_move
    else:
        min_eval = math.inf
        for move in moves:
            i, j = move
            board[i][j] = opponent
            eval, _ = count_nodes_minimax(board, depth - 1, alpha, beta, True, player, node_counter)
            board[i][j] = ' '
            if eval < min_eval:
                min_eval = eval
                best_move = move
            beta = min(beta, eval)
            if beta <= alpha:
                break
        return min_eval, best_move
    
# Bảng kết quả so sánh
def test_move_ordering(board_size, use_ordering):
    board = create_empty_board(board_size)
    node_counter = [0]
    start = time.time()

    if use_ordering:
        result = count_nodes_minimax(board, depth=4, alpha=-math.inf, beta=math.inf,
                                     maximizing_player=True, player='X', node_counter=node_counter)
    else:
        # Tạm thời tắt order_moves bằng cách thay thế bằng get_valid_moves trực tiếp
        original_order_moves = order_moves
        def no_order_moves(board, moves): return moves
        globals()['order_moves'] = no_order_moves

        result = count_nodes_minimax(board, depth=4, alpha=-math.inf, beta=math.inf,
                                     maximizing_player=True, player='X', node_counter=node_counter)

        globals()['order_moves'] = original_order_moves

    end = time.time()
    return node_counter[0], end - start

sizes = [3, 4, 5]
print("| Board | Ordering | Nodes | Time (s) |")
print("|-------|----------|-------|----------|")
for size in sizes:
    nodes1, time1 = test_move_ordering(size, use_ordering=False)
    nodes2, time2 = test_move_ordering(size, use_ordering=True)
    print(f"| {size}x{size} | No       | {nodes1}   | {time1:.4f} |")
    print(f"| {size}x{size} | Ordered  | {nodes2}   | {time2:.4f} |")



| Board | Ordering | Nodes | Time (s) |
|-------|----------|-------|----------|
| 3x3 | No       | 492   | 0.0271 |
| 3x3 | Ordered  | 206   | 0.0119 |
| 4x4 | No       | 1478   | 0.0630 |
| 4x4 | Ordered  | 801   | 0.0365 |
| 5x5 | No       | 4456   | 0.2916 |
| 5x5 | Ordered  | 1900   | 0.1038 |


Move ordering giúp giảm số lượng node duyệt đáng kể.

Thời gian xử lý cũng giảm, đặc biệt khi kích thước bàn cờ lớn hơn.

Đây là một kỹ thuật đơn giản nhưng rất hiệu quả để tăng tốc Alpha-Beta Pruning.

### The first few moves

Start with an empty board. This is the worst case scenario for minimax search with alpha-beta pruning since it needs solve all possible games that can be played (minus some pruning) before making the decision. What can you do? 

Khi bắt đầu từ một bàn cờ trống hoàn toàn (ví dụ 3x3), agent sử dụng Minimax với Alpha-Beta Pruning phải xem xét tất cả các chuỗi nước đi có thể xảy ra trong toàn bộ trò chơi để đưa ra quyết định đầu tiên. Đây là trường hợp tồi tệ nhất vì:

- Không có thông tin chiến lược ban đầu.

- Số lượng trạng thái tăng theo cấp số nhân.

- Alpha-Beta Pruning chưa thể cắt tỉa hiệu quả vì chưa có nhánh nào được đánh giá.

Để cải thiện hiệu suất trong những nước đi đầu tiên, có thể áp dụng các chiến lược sau:

1. Giới hạn độ sâu (depth)
- Giảm depth ở những lượt đầu để tránh duyệt toàn bộ cây.
- Sau vài lượt, tăng depth khi số lượng trạng thái giảm.

In [54]:
# Your code/ answer goes here.
def dynamic_depth(board):
    empty = sum(cell == ' ' for row in board for cell in row)
    if empty >= 7:
        return 3
    elif empty >= 5:
        return 5
    else:
        return 7


2. Dùng chiến lược mở đầu (opening book)
- Với các bàn cờ nhỏ như 3x3, có thể lưu sẵn một số nước đi mở đầu tốt (ví dụ: chọn trung tâm hoặc góc).
- Agent có thể chọn từ danh sách này thay vì chạy Minimax.

In [55]:
def opening_move(board, player):
    size = len(board)
    if all(cell == ' ' for row in board for cell in row):
        return (size // 2, size // 2)  # chọn ô trung tâm
    return alphabeta_agent(board, player)


3. Dùng hàm đánh giá heuristic
- Thay vì đợi đến trạng thái thắng/thua, ta có thể đánh giá các thế cờ tiềm năng ngay từ đầu.
- Điều này giúp agent đưa ra quyết định nhanh hơn và chiến lược hơn.

### Playtime

Let the Minimax Search agent play a random agent on a small board. Analyze wins, losses and draws.

Agent ngẫu nhiên

In [56]:
def random_agent(board, player):
    return random.choice(get_valid_moves(board))

Hàm chơi một ván

In [57]:
def play_game(minimax_first=True):
    board = create_empty_board(3)
    current_player = 'X' if minimax_first else 'O'
    agents = {
        'X': alphabeta_agent if minimax_first else random_agent,
        'O': random_agent if minimax_first else alphabeta_agent
    }

    while not is_terminal(board):
        move = agents[current_player](board, current_player)
        i, j = move
        board[i][j] = current_player
        current_player = 'O' if current_player == 'X' else 'X'

    return get_winner(board)


Chạy nhiều ván và thống kê

In [61]:
# Your code/ answer goes here.
results = {'Minimax Wins': 0, 'Random Wins': 0, 'Draws': 0}
num_games = 50

for _ in range(num_games):
    winner = play_game(minimax_first=True)
    if winner == 'X':
        results['Minimax Wins'] += 1
    elif winner == 'O':
        results['Random Wins'] += 1
    else:
        results['Draws'] += 1

print(f"Tổng số ván: {num_games}")
for key, value in results.items():
    print(f"{key}: {value}")


Tổng số ván: 50
Minimax Wins: 47
Random Wins: 0
Draws: 3


- Agent Minimax thắng phần lớn các ván, chứng minh khả năng ra quyết định chiến lược.

## Task 4: Heuristic Alpha-Beta Tree Search [30 points] 

### Heuristic evaluation function

Define and implement a heuristic evaluation function.

In [65]:
# Your code/ answer goes here.
def heuristic_evaluate(board, player):
    opponent = 'O' if player == 'X' else 'X'
    size = len(board)
    score = 0

    lines = []

    # Hàng và cột
    for i in range(size):
        lines.append(board[i])  # hàng
        lines.append([board[j][i] for j in range(size)])  # cột

    # Đường chéo
    lines.append([board[i][i] for i in range(size)])
    lines.append([board[i][size - 1 - i] for i in range(size)])

    for line in lines:
        if opponent not in line:
            count = line.count(player)
            score += 10 ** count  # càng nhiều quân, càng nhiều điểm
        elif player not in line:
            count = line.count(opponent)
            score -= 10 ** count  # trừ điểm nếu đối thủ có thế mạnh

    return score


Nếu một dòng có 2 quân của mình và 1 ô trống → score += 100

Nếu một dòng có 2 quân của đối thủ → score -= 100

Nếu dòng bị chặn (có cả 'X' và 'O') → không tính

Tích hợp vào minimax, thay thế hàm evaluate(board, player) trong Minimax bằng:

In [66]:
def evaluate1(board, player):
    winner = get_winner(board)
    if winner == player:
        return 10000
    elif winner is not None:
        return -10000
    else:
        return heuristic_evaluate(board, player)

def minimax2(board, depth, alpha, beta, maximizing_player, player):
    """Thuật toán minimax với alpha-beta pruning."""
    opponent = 'O' if player == 'X' else 'X'

    if depth == 0 or is_terminal(board):
        return evaluate1(board, player), None

    best_move = None
    if maximizing_player:
        max_eval = -math.inf
        for move in order_moves(board, get_valid_moves(board)):
            i, j = move
            board[i][j] = player
            eval, _ = minimax1(board, depth - 1, alpha, beta, False, player)
            board[i][j] = ' '
            if eval > max_eval:
                max_eval = eval
                best_move = move
            alpha = max(alpha, eval)
            if beta <= alpha:
                break
        return max_eval, best_move
    else:
        min_eval = math.inf
        for move in order_moves(board, get_valid_moves(board)):
            i, j = move
            board[i][j] = opponent
            eval, _ = minimax1(board, depth - 1, alpha, beta, True, player)
            board[i][j] = ' '
            if eval < min_eval:
                min_eval = eval
                best_move = move
            beta = min(beta, eval)
            if beta <= alpha:
                break
        return min_eval, best_move

Heuristic giúp agent đưa ra quyết định tốt hơn ở các trạng thái chưa kết thúc.

Có thể mở rộng để đánh giá vị trí ô (trung tâm, góc), hoặc thế “fork”.

Kết hợp với Alpha-Beta Pruning, agent sẽ nhanh hơn và thông minh hơn.

### Cutting off search 

Modify your Minimax Search with Alpha-Beta Pruning to cut off search at a specified depth and use the heuristic evaluation function. Experiment with different cutoff values.

Mục tiêu
- Sửa đổi thuật toán Minimax để dừng tìm kiếm ở độ sâu xác định (cutoff depth).

- Khi đạt đến độ sâu đó, sử dụng hàm đánh giá heuristic thay vì tiếp tục duyệt cây.

- Thử nghiệm với các giá trị depth khác nhau để xem ảnh hưởng đến hiệu suất và chất lượng nước đi.

In [67]:
# Your code/ answer goes here.
# Minimax với cutoff và heuristic
def minimax_cutoff(board, depth, alpha, beta, maximizing_player, player, cutoff_depth):
    opponent = 'O' if player == 'X' else 'X'

    if depth == 0 or is_terminal(board):
        return heuristic_evaluate(board, player), None

    best_move = None
    moves = order_moves(board, get_valid_moves(board))

    if maximizing_player:
        max_eval = -math.inf
        for move in moves:
            i, j = move
            board[i][j] = player
            eval, _ = minimax_cutoff(board, depth - 1, alpha, beta, False, player, cutoff_depth)
            board[i][j] = ' '
            if eval > max_eval:
                max_eval = eval
                best_move = move
            alpha = max(alpha, eval)
            if beta <= alpha:
                break
        return max_eval, best_move
    else:
        min_eval = math.inf
        for move in moves:
            i, j = move
            board[i][j] = opponent
            eval, _ = minimax_cutoff(board, depth - 1, alpha, beta, True, player, cutoff_depth)
            board[i][j] = ' '
            if eval < min_eval:
                min_eval = eval
                best_move = move
            beta = min(beta, eval)
            if beta <= alpha:
                break
        return min_eval, best_move
    
# Agent sử dụng minimax với cutoff và heuristic
def heuristic_agent(board, player, cutoff_depth=4):
    _, move = minimax_cutoff(board, depth=cutoff_depth, alpha=-math.inf, beta=math.inf,
                             maximizing_player=True, player=player, cutoff_depth=cutoff_depth)
    if move is None:
        move = random.choice(get_valid_moves(board))
    return move

# Thử nghiệm với các giá trị cutoff
import time

for cutoff in [2, 4, 6]:
    board = create_empty_board(3)
    start = time.time()
    move = heuristic_agent(board, 'X', cutoff_depth=cutoff)
    end = time.time()
    print(f"Cutoff depth = {cutoff} → Move: {move} | Time: {end - start:.4f} giây")



Cutoff depth = 2 → Move: (1, 1) | Time: 0.0007 giây
Cutoff depth = 4 → Move: (1, 1) | Time: 0.0064 giây
Cutoff depth = 6 → Move: (1, 1) | Time: 0.0478 giây


How many nodes are searched and how long does it take to make a move? Start with a smaller board with 4 columns and make the board larger by adding columns.

Mục tiêu
- Bắt đầu với bàn cờ nhỏ (ví dụ 3x4), sau đó tăng dần số cột (4x4, 4x5, 4x6…).

- Đo:

    - Số lượng node được duyệt trong Minimax với Alpha-Beta Pruning.

    - Thời gian tính toán để chọn nước đi.

- Phân tích ảnh hưởng của kích thước bàn cờ đến hiệu suất thuật toán.

In [68]:
# Your code/ answer goes here.
def count_nodes_minimax_cutoff(board, depth, alpha, beta, maximizing_player, player, node_counter):
    """Minimax có cutoff và đếm số node duyệt."""
    node_counter[0] += 1
    opponent = 'O' if player == 'X' else 'X'

    if depth == 0 or is_terminal(board):
        return heuristic_evaluate(board, player), None

    best_move = None
    moves = order_moves(board, get_valid_moves(board))

    if maximizing_player:
        max_eval = -math.inf
        for move in moves:
            i, j = move
            board[i][j] = player
            eval, _ = count_nodes_minimax_cutoff(board, depth - 1, alpha, beta, False, player, node_counter)
            board[i][j] = ' '
            if eval > max_eval:
                max_eval = eval
                best_move = move
            alpha = max(alpha, eval)
            if beta <= alpha:
                break
        return max_eval, best_move
    else:
        min_eval = math.inf
        for move in moves:
            i, j = move
            board[i][j] = opponent
            eval, _ = count_nodes_minimax_cutoff(board, depth - 1, alpha, beta, True, player, node_counter)
            board[i][j] = ' '
            if eval < min_eval:
                min_eval = eval
                best_move = move
            beta = min(beta, eval)
            if beta <= alpha:
                break
        return min_eval, best_move

# Thử nghiệm với các kích thước bàn cờ
def create_board(rows, cols):
    return [[' ' for _ in range(cols)] for _ in range(rows)]

print("| Board Size | Nodes Searched | Time (s) |")
print("|------------|----------------|----------|")

for cols in range(4, 8):  # từ 4 đến 7 cột
    board = create_board(4, cols)
    node_counter = [0]
    start = time.time()
    _, move = count_nodes_minimax_cutoff(board, depth=3, alpha=-math.inf, beta=math.inf,
                                         maximizing_player=True, player='X', node_counter=node_counter)
    end = time.time()
    print(f"| 4x{cols}      | {node_counter[0]}           | {end - start:.4f} |")


| Board Size | Nodes Searched | Time (s) |
|------------|----------------|----------|
| 4x4      | 344           | 0.0037 |
| 4x5      | 510           | 0.0137 |
| 4x6      | 708           | 0.0125 |
| 4x7      | 938           | 0.0234 |


### Playtime

Let two heuristic search agents (different cutoff depth, different heuristic evaluation function) compete against each other on a reasonably sized board. Since there is no randomness, you only need to let them play once.

In [77]:
# Your code/ answer goes here.

# 2 hàm heuristic khác nhau
def heuristic_v1(board, player):
    # Đơn giản: đếm số dòng/cột/chéo chưa bị chặn
    opponent = 'O' if player == 'X' else 'X'
    score = 0
    size = len(board)
    lines = []

    for i in range(size):
        lines.append(board[i])
        lines.append([board[j][i] for j in range(size)])

    lines.append([board[i][i] for i in range(size)])
    lines.append([board[i][size - 1 - i] for i in range(size)])

    for line in lines:
        if opponent not in line:
            score += line.count(player)
    return score

def heuristic_v2(board, player):
    # Nâng cao: thưởng theo số lượng quân liên tiếp
    opponent = 'O' if player == 'X' else 'X'
    score = 0
    size = len(board)
    lines = []

    for i in range(size):
        lines.append(board[i])
        lines.append([board[j][i] for j in range(size)])

    lines.append([board[i][i] for i in range(size)])
    lines.append([board[i][size - 1 - i] for i in range(size)])

    for line in lines:
        if opponent not in line:
            count = line.count(player)
            score += 10 ** count
        elif player not in line:
            count = line.count(opponent)
            score -= 10 ** count
    return score

# Agent sử dụng heuristic và cutoff
def custom_agent(board, player, cutoff_depth, heuristic_fn):
    def minimax_custom(board, depth, alpha, beta, maximizing_player):
        if depth == 0 or is_terminal(board):
            return heuristic_fn(board, player), None

        best_move = None
        moves = order_moves(board, get_valid_moves(board))
        opponent = 'O' if player == 'X' else 'X'

        if maximizing_player:
            max_eval = -math.inf
            for move in moves:
                i, j = move
                board[i][j] = player
                eval, _ = minimax_custom(board, depth - 1, alpha, beta, False)
                board[i][j] = ' '
                if eval > max_eval:
                    max_eval = eval
                    best_move = move
                alpha = max(alpha, eval)
                if beta <= alpha:
                    break
            return max_eval, best_move
        else:
            min_eval = math.inf
            for move in moves:
                i, j = move
                board[i][j] = opponent
                eval, _ = minimax_custom(board, depth - 1, alpha, beta, True)
                board[i][j] = ' '
                if eval < min_eval:
                    min_eval = eval
                    best_move = move
                beta = min(beta, eval)
                if beta <= alpha:
                    break
            return min_eval, best_move

    _, move = minimax_custom(board, cutoff_depth, -math.inf, math.inf, True)
    if move is None:
        move = random.choice(get_valid_moves(board))
    return move

# Đấu một ván
def play_custom_match(board_size=4):
    board = create_empty_board(board_size)
    current_player = 'X'
    agents = {
        'X': lambda b, p: custom_agent(b, p, cutoff_depth=3, heuristic_fn=heuristic_v1),
        'O': lambda b, p: custom_agent(b, p, cutoff_depth=5, heuristic_fn=heuristic_v2)
    }

    while not is_terminal(board):
        move = agents[current_player](board, current_player)
        i, j = move
        board[i][j] = current_player
        current_player = 'O' if current_player == 'X' else 'X'

    winner = get_winner(board)
    print("Kết quả trận đấu:")
    for row in board:
        print(row)
    print("Người thắng:", winner if winner else "Hòa")

play_custom_match(board_size=5)


Kết quả trận đấu:
['O', 'X', 'O', 'O', 'O']
['O', 'X', 'X', 'X', 'O']
['X', 'X', 'X', 'X', 'O']
[' ', 'X', 'X', 'X', 'O']
['O', 'O', ' ', ' ', 'O']
Người thắng: O


Nhận xét
- Agent 'X' dùng heuristic đơn giản và độ sâu thấp hơn.

- Agent 'O' dùng heuristic nâng cao và độ sâu lớn hơn.

- Kết quả phản ánh sự khác biệt về chiến lược và khả năng phân tích.